In [1]:
import torch

In [5]:
!pip install git+https://github.com/huggingface/transformers accelerate bitsandbytes langchain sentence-transformers faiss-cpu

  Cloning https://github.com/huggingface/transformers to /private/var/folders/cs/hh3_26_57sx7l8sdk574zbdh0000gn/T/pip-req-build-e691rcke
  Running command git clone -q https://github.com/huggingface/transformers /private/var/folders/cs/hh3_26_57sx7l8sdk574zbdh0000gn/T/pip-req-build-e691rcke
  Resolved https://github.com/huggingface/transformers to commit a1a7454107e5f005dc55c3f702d96517ad0beca4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
You should consider upgrading via the '/Users/SamuelLP/Desktop/git/projet_nlp/.nlp_venv/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import transformers
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain, RetrievalQA, ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.vectorstores.faiss import FAISS
from langchain_core.documents import Document
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
import pandas as pd
from langchain_community.embeddings.openai import OpenAIEmbeddings
device = 'cuda' if torch.cuda.is_available() else 'cpu'

ModuleNotFoundError: No module named 'transformers.utils'

In [ ]:
model = transformers.AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")# , device_map='auto')
tokenizer = transformers.AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

Loading checkpoint shards: 100%|██████████| 2/2 [01:11<00:00, 35.98s/it]


In [ ]:
import pandas as pd

train = pd.read_csv("../datas/passage_date.csv")
# test = pd.read_csv("../datas/test_data.csv")

In [ ]:
def remove_newlines(df):
    df = df.replace("\n", '', regex=True)
    return df
train = remove_newlines(train)
# test = remove_newlines(test)

In [ ]:
train.head()

,ID,filename,texte,sexe,date_accident,date_consolidation
0,0,Agen_100515.txt,Le : 12/11/2019 Cour d’appel d’Agen chambre ...,homme,1991-04-09,n.c.
1,1,Agen_1100752.txt,Le : 12/11/2019 Cour d’appel d’Agen chambre ...,homme,2005-06-10,2010-01-19
2,2,Agen_1613.txt,Le : 12/11/2019 Cour d’appel d’Agen Audience...,femme,1997-09-26,n.c.
3,3,Agen_2118.txt,Le : 12/11/2019 Cour d’appel d’Agen Audience...,femme,1982-08-07,1982-11-07
4,4,Agen_21229.txt,Le : 12/11/2019 Cour d’appel d’Agen Audience...,homme,1996-11-26,n.c.


In [ ]:
texte = train["texte"].tolist()

# RAG pipeline

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt') # Décommente cette ligne si 'punkt' n'a pas déjà été téléchargé

def preprocess_documents(df_or_series):
    pages_content = []
    
    # Vérifie si l'objet est une Series et le convertit en DataFrame si nécessaire
    if isinstance(df_or_series, pd.Series):
        df = df_or_series.to_frame().transpose()
    else:
        df = df_or_series
    
    for index, row in df.iterrows():
        texte = row["texte"]
        name = row["filename"]
        
        # Découpe le texte en phrases
        phrases = sent_tokenize(texte)
        
        for phrase in phrases:
            texte_doc = Document(page_content=phrase,
                                 metadata={'name': name})
            pages_content.append(texte_doc)
        
    return pages_content

[nltk_data] Downloading package punkt to /Users/SamuelLP/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
content = preprocess_documents(train.iloc[3,:])

In [ ]:
embedding_model = SentenceTransformerEmbeddings(model_name='almanach/camembert-base')

In [ ]:
faiss_index = FAISS.from_documents(content, embedding_model)

In [ ]:
retriever = faiss_index.as_retriever(search_kwargs={'k': 6})

In [ ]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0,
    repetition_penalty=1.2,
    return_full_text=True,
    max_new_tokens=2000)

In [ ]:
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
qa = ConversationalRetrievalChain.from_llm(llm=mistral_llm,
                                           retriever=retriever,
                                           return_source_documents=True,
                                           verbose=False)

In [17]:
query = "Quelle est la date de consolidation?"
# queries = ["Quel est le sexe de la victime?", "Quel est la date de l'accident?", "Quelle est la date de consolidation?"]
queries = ["Quelle est la date d'accident?"]

In [18]:

for query in queries :

    if query == "Quel est le sexe de la victime?":
        result = qa({"question" : f"""Tu es un assistant juridique. Tu vas recevoir plusieurs textes juridiques et tu vas devoir répondre à la recherché une ou des informations demandée(s) par l'utilisateur à partir des documents données.
    On va te demander les informations suivantes : Sexe de la victime, date de l'accident, date de consolidation.
    Le sexe de la victime est toujours contenu dans le document et ne peux prendre que deux valeurs : "homme" ou "femme".
    voici la demande : {query} """,
    "chat_history": []})

    elif query == "Quelle est la date de l'accident?":
        result = qa({"question" : f"""Tu es un assistant juridique. Tu vas recevoir plusieurs textes juridiques et tu vas devoir répondre à la recherché une ou des informations demandée(s) par l'utilisateur à partir des documents données.
    On va te demander les informations suivantes : Sexe de la victime, date de l'accident, date de consolidation.
    La date de l'accident correspond à la date à laquelle l'accident s'est produit. Elle est toujours quelque part dans le document (généralement au début) sauf dans de très rares cas.. Nous attendons une date au format jj/mm/aaaa.
    voici la demande : {query} """,
    "chat_history": []})

    elif query == "Quelle est la date de consolidation?":
        result = qa({"question" : f"""Tu es un assistant juridique. Tu vas recevoir plusieurs textes juridiques et tu vas devoir répondre à la recherché une ou des informations demandée(s) par l'utilisateur à partir des documents données.
    On va te demander les informations suivantes : Sexe de la victime, date de l'accident, date de consolidation.
    La date de consolidation est la date à laquelle les blessures de la victime sont devenues stables et ont été déclarées définitives par un médecin. L'information devrait être présente dans la plupart des cas mais parfois elle est soit manquante (tu afficheras la valeur "n.c.") soit non applicable (tu afficheras la valeur "n.a.") si la blessure n'a pas stabilisé avant le décès de la victime.
    voici la demande : {query} """,
    "chat_history": []})


/Users/SamuelLP/Desktop/git/projet_nlp/.nlp_venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/Users/SamuelLP/Desktop/git/projet_nlp/.nlp_venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [19]:
for element in result["source_documents"]:
    source = f'\n- {element.metadata["name"]}'
    print(source)


- Agen_2118.txt

- Agen_2118.txt

- Agen_2118.txt

- Agen_2118.txt

- Agen_2118.txt

- Agen_2118.txt


In [20]:
print(result['answer'])

 The information about the victim's gender is not provided in the given texts.


In [21]:
source

'\n- Agen_2118.txt'